# Libraries


In [2]:
import random
import numpy as np

# Constants, params


In [11]:
BOARD_SIZE = 16
POPULATION_SIZE = 1000
MUTATION_RATE = 0.1
MAX_GENERATION = 10000

# ChessBoard class (Visualize the Chessboard in 2D array)

In [4]:
class ChessBoard:
  """ A class to represent a chessboard in 2D array.

  Attributes:
    board_size (int): The size of the chessboard.
    board (numpy.ndarray): A 2D array representing the chessboard.

  Every cell of the initial chessboard will be filled by a dot "."
  And the Queen will be perform by a "Q" after process the `state`.

  """

  def __init__ (self, board_size:int):
    self.board_size = board_size
    self.board = np.full((self.board_size, self.board_size), ".", dtype='<U1')

  def place_queen(self, state):
    self.board.fill(".")
    for col, row in enumerate(state):
      if row >= 0 and row < self.board_size:
        self.board[row, col] = "Q"

  def print_board(self):
    for row in self.board:
      print(" ".join(row))

# State class (Visualize the Chess board in 1D array)

## For example:
- We have a 4x4 Chess board below (Index of colum and row both start from 0):
```
\ 0 1 2 3
0 . . . Q
1 Q . . .
2 . Q . .
3 . . Q .
```
It will be visualized: `[1, 2, 3, 0]`
- That means, `col[0]` have a Queen placed at `row[1]` -> `queen[1][0]` and so on ...

In [5]:
class State:
  """ A class to represent a state of a chessboard.

  Attributes:
    board_size (int): The size of the chessboard.
    state (list): A list of integers representing the row positions of queens.
    maximum_non_attacking_pairs (int): The maximum number of non-attacking pairs of queens.
    fitness_score (int): The fitness score of the state.
    selection_prob (float): The selection probability of the state.
    length (int): The length of the state
  """

  def __init__(self, board_size:int, random_state:bool):
    self.board_size = board_size
    self.state = []
    self.maximum_non_attacking_pairs = self.board_size * (self.board_size - 1) // 2
    self.selection_prob = None

    if random_state:
      self.state = [random.randint(0, self.board_size) for _ in range(0, self.board_size)]
      self.fitness_score = self.calculate_fitness(self.board_size)
      self.length = self.get_state_length()

  def set_state(self, state):
    self.state = state
    self.fitness_score = self.calculate_fitness(self.board_size)
    self.length = self.get_state_length()

  def get_state(self):
    return self.state

  def set_selection_prob(self, selection_prob):
    self.selection_prob = selection_prob

  def get_selection_prob(self):
    return self.selection_prob

  def get_fitness(self):
    return self.fitness_score

  def calculate_fitness(self, board_size):
    pair_of_attacking_queens = 0
    for i in range(board_size):
      for j in range(i + 1, board_size):
        if self.state[i] == self.state[j] or abs(i - j) == abs(self.state[i] - self.state[j]):
          pair_of_attacking_queens += 1

    return self.maximum_non_attacking_pairs - pair_of_attacking_queens

  def mutate_state(self, position: int, value: int):
    self.state[position - 1] = value
    self.fitness_score = self.calculate_fitness(self.board_size)

  def get_state_length(self):
    return __builtins__.len(self.state)

  def to_table(self, table_name:str):
    print(f"{table_name.upper()}")
    print(f"{'State':<65} {'Length':<15} {'Fitness Score':<15} {'Selection Probability' :<15}")
    print("=" * 119)
    print(f"{str(self.get_state()):<67} {self.get_state_length(): <18} {self.get_fitness():<18} {self.get_selection_prob()}")

# Testing State class and ChessBoard class

In [7]:
if __name__ == "__main__":
  state_obj = State(board_size=BOARD_SIZE, random_state=True)
  state_obj.set_state([5, 15, 2, 11, 6, 16, 3, 10, 7, 4, 14, 1, 13, 9, 12, 8])
  state_obj.to_table("\nSolution")

  chess_board_obj = ChessBoard(board_size=BOARD_SIZE)
  chess_board_obj.place_queen(state_obj.get_state())
  chess_board_obj.print_board()

  state_obj.set_state([9, 6, 8, 1, 12, 15, 7, 10, 3, 16, 4, 11, 0, 2, 5, 14])
  state_obj.to_table("\nSolution")
  chess_board_obj.place_queen(state_obj.get_state())
  chess_board_obj.print_board()

  state_obj.set_state([12, 3, 0, 7, 11, 2, 15, 1, 14, 5, 9, 6, 13, 16, 4, 8])
  state_obj.to_table("\nSolution")
  chess_board_obj.place_queen(state_obj.get_state())
  chess_board_obj.print_board()

  state_obj.set_state([0, 4, 12, 10, 5, 13, 15, 9, 2, 6, 1, 16, 7, 11, 3, 8])
  state_obj.to_table("\nSolution")
  chess_board_obj.place_queen(state_obj.get_state())
  chess_board_obj.print_board()

  state_obj.set_state([7, 14, 2, 8, 13, 11, 4, 1, 5, 10, 12, 15, 0, 16, 9, 6])
  state_obj.to_table("\nSolution")
  chess_board_obj.place_queen(state_obj.get_state())
  chess_board_obj.print_board()

  # chess_board_obj.print_board()



SOLUTION
State                                                             Length          Fitness Score   Selection Probability
[5, 15, 2, 11, 6, 16, 3, 10, 7, 4, 14, 1, 13, 9, 12, 8]             16                 120                None
. . . . . . . . . . . . . . . .
. . . . . . . . . . . Q . . . .
. . Q . . . . . . . . . . . . .
. . . . . . Q . . . . . . . . .
. . . . . . . . . Q . . . . . .
Q . . . . . . . . . . . . . . .
. . . . Q . . . . . . . . . . .
. . . . . . . . Q . . . . . . .
. . . . . . . . . . . . . . . Q
. . . . . . . . . . . . . Q . .
. . . . . . . Q . . . . . . . .
. . . Q . . . . . . . . . . . .
. . . . . . . . . . . . . . Q .
. . . . . . . . . . . . Q . . .
. . . . . . . . . . Q . . . . .
. Q . . . . . . . . . . . . . .

SOLUTION
State                                                             Length          Fitness Score   Selection Probability
[9, 6, 8, 1, 12, 15, 7, 10, 3, 16, 4, 11, 0, 2, 5, 14]              16                 120                None
. . . 

# GeneticAlgorithm class

In [12]:
class Population:
  def __init__(self, board_size:int, population_size:int, mutation_rate:float, none_population: bool):
    self.board_size = board_size
    self.population_size = population_size
    self.mutation_rate = mutation_rate
    if none_population:
      self.population = []
    else:
      self.population = self.generate_population()

    self.max_fitness = self.calculate_max_fitness()
    self.selection_prob = self.set_selection_prob_for_all_state()

  def add_new_state_to_population(self, state):
    self.population.append(state)

  def generate_population(self):
    return [State(self.board_size, random_state=True) for _ in range(self.population_size)]

  def print_population(self):
    for state in self.population:
      print(f"{str(state.get_state()):<70} {state.get_fitness():<18}")

  def get_population(self):
    return self.population

  def get_population_length(self):
    return __builtins__.len(self.population)

  def set_selection_prob_for_all_state(self):
    total_fitness = sum(state.get_fitness() for state in self.population)
    selection_probs = []
    for state in self.population:
      selection_prob = state.get_fitness() / total_fitness
      state.set_selection_prob(selection_prob)
      selection_probs.append(selection_prob)

    return selection_probs

  def evaluate_population(self):
    self.population.sort(key=lambda state: state.get_fitness(), reverse=True)
    self.population = self.population[:-1]

  def make_couples(self):
    if (len(self.population) % 2) != 0:
      self.population.append(self.population[0])

  def population_pop(self, pos:int):
    self.population.pop(pos)

  def crossover_two_state(self, parent1, parent2):
    crossover_start_point = random.randint(1, self.board_size - 3)
    crossover_end_point = random.randint(crossover_start_point + 1, self.board_size - 1)

    allele1 = parent1.get_state()[crossover_start_point:crossover_end_point]
    allele3 = parent2.get_state()[crossover_start_point:crossover_end_point]

    allele2 = parent2.get_state().copy()
    for num in allele1:
      for i in allele2:
        if num == i:
          allele2.remove(i)
          break

    allele2 = allele2

    allele4 = parent1.get_state().copy()
    for num in allele3:
      for i in allele4:
        if num == i:
          allele4.remove(i)
          break

    allele4 = allele4

    tmp1 = __builtins__.len(allele1) + __builtins__.len(allele2)
    tmp2 = __builtins__.len(allele3) + __builtins__.len(allele4)

    if tmp1 == 16:
      child1 = allele2[:crossover_start_point] + allele1 + allele2[crossover_start_point:]
    else:
      child_end_point = -(abs(BOARD_SIZE - tmp1))
      child1 = allele2[:crossover_start_point] + allele1 + allele2[crossover_start_point:child_end_point]

    if tmp2 == 16:
      child2 = allele4[:crossover_start_point] + allele3 + allele4[crossover_start_point:]
    else:
      child_end_point = -(abs(BOARD_SIZE - tmp2))
      child2 = allele4[:crossover_start_point] + allele3 + allele4[crossover_start_point:child_end_point]

    child1_state = State(self.board_size, random_state=False)
    child1_state.set_state(child1)

    child2_state = State(self.board_size, random_state=False)
    child2_state.set_state(child2)

    return child1_state, child2_state

  def crossover_population(self):
    for i in range(0, len(self.population), 2):
      parent1 = self.population[i]
      parent2 = self.population[i + 1]
      child1, child2 = self.crossover_two_state(parent1, parent2)

      self.population[i] = child1
      self.population[i + 1] = child2

  def mutation(self):
    for state in self.population:
        if random.random() < self.mutation_rate:
            random_position = random.randint(0, self.board_size - 1)
            random_value = random.randint(0, self.board_size - 1)
            state.mutate_state(random_position, random_value)

  def calculate_max_fitness(self):
    max_fitness = 0
    for state in self.population:
      if state.get_fitness() > max_fitness:
        max_fitness = state.get_fitness()
    return max_fitness

  def get_state_max_fitness(self):
    for state in self.population:
      if state.get_fitness() == self.max_fitness:
        return state

  def get_max_fitness(self):
    self.max_fitness = self.calculate_max_fitness()
    return self.max_fitness

  def to_table(self, table_name:str):
    print(f"{table_name.upper()} (Length = {self.get_population_length()})")
    print("=" * 103)
    print(f"{'State':<65} {'Fitness Score':<15} {'Selection Probability' :<15}")
    print("=" * 103)
    for state in self.population:
      print(f"{str(state.get_state()):<70} {state.get_fitness():<17} {state.get_selection_prob():.4f}")

if __name__ == "__main__":
  current_generation = Population(board_size=BOARD_SIZE, population_size=POPULATION_SIZE, mutation_rate=MUTATION_RATE, none_population=False)
  current_generation.to_table("\nInitial Population")

  loop = 0

  solutions = []

  def run(current_generation):
    global loop
    while loop < MAX_GENERATION:
      print(f"LOOP: {loop} / {MAX_GENERATION} ", end="")
      current_generation.evaluate_population()
      current_generation.set_selection_prob_for_all_state()
      # current_generation.to_table("\nAfter evaluation")
      current_generation.make_couples()
      current_generation.set_selection_prob_for_all_state()
      # current_generation.to_table("\nAfter make couple")
      current_generation.crossover_population()
      current_generation.set_selection_prob_for_all_state()
      # current_generation.to_table("\nAfter crossover")
      current_generation.mutation()
      current_generation.set_selection_prob_for_all_state()
      # current_generation.to_table("\nAfter mutation")

      print(f"--> Max fitness: {current_generation.get_max_fitness()}")

      if current_generation.get_max_fitness() == 120:
        print(f"\nSOLUTION FOUND at loop {loop} / {MAX_GENERATION}")
        solution = current_generation.get_state_max_fitness()
        # solution.totable("SOLUTION")
        print(solution.get_state())

        if solution not in solutions:
          solutions.append(solution.get_state().copy())

      loop += 1

  run(current_generation=current_generation)

  if solutions is not None:
    print("\n\nSOLUTIONS:")
    for solution in solutions:
      print(solution)
  else:
    print("No solutions found.")

Streaming output truncated to the last 5000 lines.
LOOP: 5012 / 10000 --> Max fitness: 116
LOOP: 5013 / 10000 --> Max fitness: 116
LOOP: 5014 / 10000 --> Max fitness: 117
LOOP: 5015 / 10000 --> Max fitness: 116
LOOP: 5016 / 10000 --> Max fitness: 118
LOOP: 5017 / 10000 --> Max fitness: 117
LOOP: 5018 / 10000 --> Max fitness: 118
LOOP: 5019 / 10000 --> Max fitness: 117
LOOP: 5020 / 10000 --> Max fitness: 116
LOOP: 5021 / 10000 --> Max fitness: 117
LOOP: 5022 / 10000 --> Max fitness: 117
LOOP: 5023 / 10000 --> Max fitness: 118
LOOP: 5024 / 10000 --> Max fitness: 116
LOOP: 5025 / 10000 --> Max fitness: 117
LOOP: 5026 / 10000 --> Max fitness: 117
LOOP: 5027 / 10000 --> Max fitness: 117
LOOP: 5028 / 10000 --> Max fitness: 117
LOOP: 5029 / 10000 --> Max fitness: 116
LOOP: 5030 / 10000 --> Max fitness: 117
LOOP: 5031 / 10000 --> Max fitness: 118
LOOP: 5032 / 10000 --> Max fitness: 117
LOOP: 5033 / 10000 --> Max fitness: 117
LOOP: 5034 / 10000 --> Max fitness: 117
LOOP: 5035 / 10000 --> Max fi

In [ ]:
lst2 = [3, 1, 6, 4, 5]
lst1 = [2, 1, 5, 8, 7]

print(lst2[-4:])

[1, 6, 4, 5]
